In [11]:
import json
import pandas as pd
from pathlib import Path

RAW = Path("../data/raw/cancer_trials_raw.json")
OUT = Path("../data/processed")
OUT.mkdir(parents=True, exist_ok=True)

with open(RAW) as f:
    studies = json.load(f)




In [12]:
print(studies[0].keys()) # Print the keys of the first study to understand the structure

dict_keys(['protocolSection', 'derivedSection', 'hasResults'])


In [13]:
#Field names under protocolSection
print(studies[0]["protocolSection"].keys())

#Field names under identificationModule
print(studies[0]["derivedSection"].keys())

#Field names under statusModule
print(studies[0]["hasResults"])


dict_keys(['identificationModule', 'statusModule', 'sponsorCollaboratorsModule', 'oversightModule', 'descriptionModule', 'conditionsModule', 'designModule', 'armsInterventionsModule', 'outcomesModule', 'eligibilityModule', 'contactsLocationsModule', 'ipdSharingStatementModule'])
dict_keys(['miscInfoModule', 'conditionBrowseModule'])
False


In [19]:
#Creating helpers for safe extraction
def safe_list(x):
    return x if isinstance(x, list) else []

def safe_dict(x):
    return x if isinstance(x, dict) else {}


# Normalize date strings to date objects
def norm_date(d):
    if not d:
        return None
    return pd.to_datetime(d, errors="coerce").date()


# Tables (lists of rows)
trials = []
conditions = []
interventions = []
arms = []
arm_interventions = []
primary_outcomes = []
secondary_outcomes = []
locations = []
sponsors = []
collaborators = []


# Parse

for idx, s in enumerate(studies, start=1):
    protocol = s.get("protocolSection", {})
    ident = protocol.get("identificationModule", {})
    status = protocol.get("statusModule", {})
    cond_mod = protocol.get("conditionsModule", {})
    design = protocol.get("designModule", {})
    arms_mod = protocol.get("armsInterventionsModule", {})
    outcomes_mod = protocol.get("outcomesModule", {})
    loc_mod = protocol.get("contactsLocationsModule", {})
    sponsor_mod = protocol.get("sponsorCollaboratorsModule", {})

    nct_id = ident.get("nctId")

    # trials (1 row per study)
    # All fields are optional except nct_id
    # Use .get to return None if field is missing
    trials.append({
        "nct_id": nct_id,
        "brief_title": ident.get("briefTitle"),
        "official_title": ident.get("officialTitle"),
        "organization": safe_dict(ident.get("organization", {})).get("fullName"),
        "overall_status": status.get("overallStatus"),
        "why_stopped": status.get("whyStopped"),
        "start_date": norm_date(safe_dict(status.get("startDateStruct", {})).get("date")),
        "primary_completion_date": norm_date(safe_dict(status.get("primaryCompletionDateStruct", {})).get("date")),
        "completion_date": norm_date(safe_dict(status.get("completionDateStruct", {})).get("date")),
        "study_type": design.get("studyType"),
        
        #phase is a list; join into a single string
        "phases": ";".join(safe_list(design.get("phases"))),
        "allocation": design.get("allocation"),
        "intervention_model": design.get("interventionModel"),
        "masking": safe_dict(design.get("maskingInfo", {})).get("masking"),
        "primary_purpose": design.get("primaryPurpose"),
    })

    #conditions (many per study)
    for c in safe_list(cond_mod.get("conditions")):
        conditions.append({"nct_id": nct_id, "condition": c})

    #interventions (many per study)
    for itv in safe_list(arms_mod.get("interventions")):
        itv = safe_dict(itv)
        interventions.append({
            "nct_id": nct_id,
            "intervention_type": itv.get("type"),
            "intervention_name": itv.get("name"),
            "description": itv.get("description"),
        })

    #arms (many per study)
    for arm in safe_list(arms_mod.get("armGroups")):
        arm = safe_dict(arm)
        arm_label = arm.get("label")
        arms.append({
            "nct_id": nct_id,
            "arm_label": arm_label,
            "arm_type": arm.get("type"),
            "description": arm.get("description"),
        })
        # which interventions are in which arm
        for nm in safe_list(arm.get("interventionNames")):
            arm_interventions.append({
                "nct_id": nct_id,
                "arm_label": arm_label,
                "intervention_name": nm
            })

    #outcomes
    for o in safe_list(outcomes_mod.get("primaryOutcomes")):
        o = safe_dict(o)
        primary_outcomes.append({
            "nct_id": nct_id,
            "outcome_measure": o.get("measure"),
            "time_frame": o.get("timeFrame"),
            "description": o.get("description"),
        })

    for o in safe_list(outcomes_mod.get("secondaryOutcomes")):
        o = safe_dict(o)
        secondary_outcomes.append({
            "nct_id": nct_id,
            "outcome_measure": o.get("measure"),
            "time_frame": o.get("timeFrame"),
            "description": o.get("description"),
        })

    for loc in safe_list(loc_mod.get("locations")):
        loc = safe_dict(loc)
        facility_val = loc.get("facility")
        if isinstance(facility_val, dict):
            facility_name = facility_val.get("name")
        else:
            facility_name = facility_val  # string or None

        locations.append({
            "nct_id": nct_id,
            "facility_name": facility_name,
            "city": loc.get("city"),
            "state": loc.get("state"),
            "country": loc.get("country"),
            "zip": loc.get("zip"),
            "status": loc.get("status"),
        })

    #sponsors / collaborators
    lead = safe_dict(sponsor_mod.get("leadSponsor", {}))
    if lead:
        sponsors.append({
            "nct_id": nct_id,
            "lead_sponsor_name": lead.get("name"),
            "lead_sponsor_class": lead.get("class"),
        })

    for col in safe_list(sponsor_mod.get("collaborators")):
        col = safe_dict(col)
        collaborators.append({
            "nct_id": nct_id,
            "collaborator_name": col.get("name"),
            "collaborator_class": col.get("class"),
        })

    if idx % 10000 == 0:
        print(f"Processed {idx} studies...")


# Drop duplicates and write tables to CSV
pd.DataFrame(trials).drop_duplicates().to_csv(OUT / "trials.csv", index=False)
pd.DataFrame(conditions).drop_duplicates().to_csv(OUT / "conditions.csv", index=False)
pd.DataFrame(interventions).drop_duplicates(["nct_id", "intervention_name", "intervention_type"]).to_csv(OUT / "interventions.csv", index=False)
pd.DataFrame(arms).drop_duplicates(["nct_id", "arm_label"]).to_csv(OUT / "arms.csv", index=False)
pd.DataFrame(arm_interventions).drop_duplicates().to_csv(OUT / "arm_interventions.csv", index=False)
pd.DataFrame(primary_outcomes).drop_duplicates(["nct_id", "outcome_measure", "time_frame"]).to_csv(OUT / "primary_outcomes.csv", index=False)
pd.DataFrame(secondary_outcomes).drop_duplicates(["nct_id", "outcome_measure", "time_frame"]).to_csv(OUT / "secondary_outcomes.csv", index=False)
pd.DataFrame(locations).drop_duplicates(["nct_id", "facility_name", "city", "country"]).to_csv(OUT / "locations.csv", index=False)
pd.DataFrame(sponsors).drop_duplicates(["nct_id", "lead_sponsor_name"]).to_csv(OUT / "sponsors.csv", index=False)
pd.DataFrame(collaborators).drop_duplicates(["nct_id", "collaborator_name"]).to_csv(OUT / "collaborators.csv", index=False)

print("Done. Wrote tables to:", OUT)

Processed 10000 studies...
Processed 20000 studies...
Processed 30000 studies...
Processed 40000 studies...
Processed 50000 studies...
Processed 60000 studies...
Processed 70000 studies...
Processed 80000 studies...
Processed 90000 studies...
Processed 100000 studies...
Processed 110000 studies...
Processed 120000 studies...
Processed 130000 studies...
Done. Wrote tables to: ../data/processed


In [17]:
loc_mod = studies[0]["protocolSection"].get("contactsLocationsModule", {})
loc = loc_mod.get("locations", [])[0]
print(loc.keys())
print(type(loc.get("facility")), loc.get("facility"))

dict_keys(['facility', 'city', 'state', 'zip', 'country', 'geoPoint'])
<class 'str'> Duke University Medical Center
